In [1]:
import os, json, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, SubsetRandomSampler, Dataset

import optuna
from metrics import metrics_from_preds
from preprocess import load_data

In [2]:
def set_seed(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.set_num_threads(1)

In [3]:
set_seed(42)

device = torch.device("cpu")  # para máxima reproducibilidad
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [4]:
vids_training, labels_training, vids_val, labels_val, vids_test, labels_test = load_data()

In [5]:
class VideoDataset(Dataset):
    def __init__(self, videos, labels):
        self.videos = videos
        self.labels = labels
    def __len__(self):
        return len(self.videos)
    def __getitem__(self, idx):
        x = torch.tensor(np.array(self.videos[idx]), dtype=torch.float32)  # (T, F)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

In [6]:
def collate_pad(batch):
    xs, ys = zip(*batch)
    x = torch.stack(xs, dim=0)  # (B, T, F)
    y = torch.stack(ys, dim=0)  # (B,)
    return x, y

In [7]:
set_seed(42)

In [8]:
train_dataset = VideoDataset(vids_training, labels_training)
val_dataset   = VideoDataset(vids_val, labels_val)

g = torch.Generator().manual_seed(42)
perm = torch.randperm(len(train_dataset), generator=g).tolist()
train_sampler = SubsetRandomSampler(perm)

train_loader_seq = DataLoader(
    train_dataset, batch_size=32, shuffle=False, sampler=train_sampler,
    num_workers=0, collate_fn=collate_pad
)
val_loader_seq = DataLoader(
    val_dataset, batch_size=32, shuffle=False,
    num_workers=0, collate_fn=collate_pad
)

In [9]:
class LSTMClassifier(nn.Module):
    """Clasificador secuencial basado en LSTM.
       Entrada:  (B, T, F)
       LSTM:     salida (B, T, hidden)
       Pooling:  último paso temporal
       Salida:   (B, num_classes)
    """
    def __init__(self, input_size=258, hidden_size=128, num_layers=2, dropout=0.1, num_classes=4, bidirectional=False):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=bidirectional
        )
        out_dim = hidden_size * (2 if bidirectional else 1)
        self.fc = nn.Linear(out_dim, num_classes)

    def forward(self, x):  # x: (B, T, F)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # último paso temporal
        return self.fc(out)

In [10]:
def optimize_lstm(train_loader_seq, val_loader_seq, n_trials=10, max_epochs=50, save_prefix="hpo_lstm"):
    set_seed(42)

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler,
                                pruner=optuna.pruners.MedianPruner())
    all_trials = []

    sample_x, _ = next(iter(train_loader_seq))
    input_size = sample_x.shape[-1]

    def objective(trial):
        hidden  = trial.suggest_categorical("hidden_size", [64, 96, 128, 160, 192])
        layers  = trial.suggest_int("num_layers", 1, 4)
        dropout = trial.suggest_float("dropout", 0.0, 0.5)
        bi      = trial.suggest_categorical("bidirectional", [False, True])
        lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
        wd      = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

        model = LSTMClassifier(
            input_size=input_size, hidden_size=hidden, num_layers=layers,
            dropout=dropout, num_classes=4, bidirectional=bi
        ).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

        best_val = 0.0
        for epoch in range(max_epochs):
            model.train()
            for xb, yb in train_loader_seq:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()

            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for xb, yb in val_loader_seq:
                    xb = xb.to(device)
                    preds = model(xb).argmax(dim=1)
                    y_true.extend(yb.numpy())
                    y_pred.extend(preds.cpu().numpy())
            val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]

            if val_acc > best_val:
                best_val = val_acc

            trial.report(val_acc, epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()

        all_trials.append({"trial": trial.number, "params": trial.params, "best_val_acc": best_val})
        return best_val

    study.optimize(objective, n_trials=n_trials)
    with open(f"{save_prefix}_all.json", "w") as f:
        json.dump(all_trials, f, indent=2)
    with open(f"{save_prefix}_best.json", "w") as f:
        json.dump({"best_params": study.best_params, "best_value": study.best_value}, f, indent=2)
    return study

In [12]:
if __name__ == "__main__":
    study_lstm = optimize_lstm(train_loader_seq, val_loader_seq, n_trials=40, max_epochs=50)
    print("Mejor resultado:", study_lstm.best_value)
    print("Mejores hiperparámetros:", study_lstm.best_params)

    # =============================================================================
    # [9] Reentrenar el mejor modelo de forma 100% reproducible
    # =============================================================================
    best = study_lstm.best_params
    set_seed(42)

    sample_x, _ = next(iter(train_loader_seq))
    input_size = sample_x.shape[-1]

    model = LSTMClassifier(
        input_size=input_size,
        hidden_size=best["hidden_size"],
        num_layers=best["num_layers"],
        dropout=best["dropout"],
        num_classes=4,
        bidirectional=best["bidirectional"]
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=best["lr"], weight_decay=best["weight_decay"])

    for epoch in range(50):
        model.train()
        for xb, yb in train_loader_seq:
            xb = xb.to(device); yb = yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for xb, yb in val_loader_seq:
                xb = xb.to(device)
                preds = model(xb).argmax(dim=1)
                y_true.extend(yb.numpy())
                y_pred.extend(preds.cpu().numpy())
        val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]
        print(f"Epoch {epoch+1:03d} | Val Acc: {val_acc:.4f}")

[I 2025-11-04 11:25:26,069] A new study created in memory with name: no-name-d54623d4-3b30-4d80-b2d5-57425c49ee63
[I 2025-11-04 11:25:42,084] Trial 0 finished with value: 0.8722222222222222 and parameters: {'hidden_size': 96, 'num_layers': 1, 'dropout': 0.02904180608409973, 'bidirectional': False, 'lr': 0.0013035123791853842, 'weight_decay': 1.3934502251337584e-10}. Best is trial 0 with value: 0.8722222222222222.
[I 2025-11-04 11:26:00,521] Trial 1 finished with value: 0.8555555555555555 and parameters: {'hidden_size': 64, 'num_layers': 2, 'dropout': 0.2623782158161189, 'bidirectional': False, 'lr': 0.0008369042894376068, 'weight_decay': 9.472334467618544e-10}. Best is trial 0 with value: 0.8722222222222222.
[I 2025-11-04 11:29:18,540] Trial 2 finished with value: 0.8611111111111112 and parameters: {'hidden_size': 160, 'num_layers': 3, 'dropout': 0.29620728443102123, 'bidirectional': True, 'lr': 0.0001096524277832185, 'weight_decay': 2.853390105240223e-10}. Best is trial 0 with value: 

Mejor resultado: 0.9
Mejores hiperparámetros: {'hidden_size': 64, 'num_layers': 2, 'dropout': 0.11159078185167515, 'bidirectional': True, 'lr': 0.0016719911260666689, 'weight_decay': 1.242881605708831e-10}
Epoch 001 | Val Acc: 0.6000
Epoch 002 | Val Acc: 0.7444
Epoch 003 | Val Acc: 0.7778
Epoch 004 | Val Acc: 0.7444
Epoch 005 | Val Acc: 0.8222
Epoch 006 | Val Acc: 0.7389
Epoch 007 | Val Acc: 0.6889
Epoch 008 | Val Acc: 0.8056
Epoch 009 | Val Acc: 0.8000
Epoch 010 | Val Acc: 0.7222
Epoch 011 | Val Acc: 0.8389
Epoch 012 | Val Acc: 0.8111
Epoch 013 | Val Acc: 0.8556
Epoch 014 | Val Acc: 0.8111
Epoch 015 | Val Acc: 0.8556
Epoch 016 | Val Acc: 0.8278
Epoch 017 | Val Acc: 0.8056
Epoch 018 | Val Acc: 0.8444
Epoch 019 | Val Acc: 0.7833
Epoch 020 | Val Acc: 0.8278
Epoch 021 | Val Acc: 0.7611
Epoch 022 | Val Acc: 0.8000
Epoch 023 | Val Acc: 0.8333
Epoch 024 | Val Acc: 0.8778
Epoch 025 | Val Acc: 0.8500
Epoch 026 | Val Acc: 0.8722
Epoch 027 | Val Acc: 0.8444
Epoch 028 | Val Acc: 0.8389
Epoch 029 